***3.0 - Chuẩn bị dữ liệu***

In [1]:
import pandas as pd
duong_dan = '../data_clean/'

df_thong_tin = pd.read_csv(duong_dan + 'sensor_info_clean.csv')
df_doc_cam_bien = pd.read_csv(duong_dan + 'sensor_readings_clean.csv')
df_bao_tri = pd.read_csv(duong_dan + 'maintenance_log_clean.csv')

df_ket_hop = df_doc_cam_bien.merge(df_thong_tin,on='sensor_id',how='left')

df_ket_hop.head()

,sensor_id,timestamp,value,unit,machine_id,location,sensor_type
0,SEN08,2024-01-08 12:23:00,NaN,°C,NaN,NaN,NaN
1,sen03,2024-01-08 18:46:00,80.0,A,NaN,NaN,NaN
2,SEN09,2024-01-01 20:33:00,25.5,mm/s,NaN,NaN,NaN
3,SEN08,2024-01-07 17:47:00,45.0,°C,NaN,NaN,NaN
4,SEN09,2024-01-09 12:37:00,25.5,A,NaN,NaN,NaN


***3.1 - Tính giá trị đo trung bình theo machine_id × sensor_type***

In [3]:
trung_binh_theo_may_loai = (df_ket_hop.groupby(['machine_id', 'sensor_type'])['value'].mean())
trung_binh_theo_may_loai

machine_id  sensor_type
MC01        Current        38.571429
Name: value, dtype: float64

***3.2 - Tổng số giờ downtime theo machine_id***

In [4]:
tong_downtime = df_bao_tri.groupby('machine_id')['downtime_hours'].sum()
tong_downtime

machine_id
MC01    26.0
MC02    22.0
MC03    30.0
MC04    24.5
MC05    21.5
Name: downtime_hours, dtype: float64

***3.3 - Xác định nhóm máy/cảm biến có nguy cơ sự cố cao***

In [5]:
so_su_co = df_bao_tri.groupby('machine_id')['issue_type'].count()
tong_downtime = df_bao_tri.groupby('machine_id')['downtime_hours'].sum()

nguy_co = pd.DataFrame({'so_su_co': so_su_co,'tong_downtime': tong_downtime}).sort_values(['so_su_co', 'tong_downtime'], ascending=False)
nguy_co

,so_su_co,tong_downtime
machine_id,,
MC01,17,26.0
MC05,12,21.5
MC02,11,22.0
MC03,10,30.0
MC04,10,24.5


***3.4 - Phân tích mối liên hệ giữa giá trị đo bất thường và downtime.***

In [8]:
df_ket_hop['bat_thuong'] = df_ket_hop['value'].isna().astype(int)

so_bat_thuong = df_ket_hop.groupby('machine_id')['bat_thuong'].sum()
tong_downtime = df_bao_tri.groupby('machine_id')['downtime_hours'].sum()

lien_he = pd.DataFrame({'so_reading_bat_thuong': so_bat_thuong,'tong_downtime': tong_downtime})
lien_he

,so_reading_bat_thuong,tong_downtime
machine_id,,
MC01,1.0,26.0
MC02,NaN,22.0
MC03,NaN,30.0
MC04,NaN,24.5
MC05,NaN,21.5
